In [1]:
from glob import glob
import math
import numpy as np
import pandas as pd
import re

In [2]:
def tokenize(txt):
    name, date, content = txt.split("\t")
    tokens = re.split("\W+", content)
    tokens = [t.lower() for t in tokens if t]
    return name, date, tokens

In [3]:
def frameforpath(filepath):
    with open(filepath) as f:
        txt = f.read()
        name, date, tokens = tokenize(txt)
        m = re.search( "([0-9]{4})", date)
        year = m.group()
        tmp = pd.DataFrame(tokens)
        tmp.columns = ['token']
        tmp['name'] = name
        tmp['year'] = year
        tmp['seq'] = tmp.index
    return tmp

In [4]:
filepaths = sorted(glob("../inauguralspeeches/*.txt"))
dframe = pd.DataFrame()
for filepath in filepaths:
    dframe = dframe.append(frameforpath(filepath))

In [5]:
tokens = dframe[['token','seq']].groupby(["token"]).count()
tokens.columns = ['count']

In [6]:
tokenyears = dframe[['token','year','seq']].groupby(["token",'year']).count()
tokenyears.columns = ['count']
termcounts = tokenyears.reset_index().pivot_table(index="token", columns='year', values="count")

In [7]:
numdocs = termcounts.shape[1]

In [8]:
doctotals = termcounts.sum()
termtotals = termcounts.sum(axis=1)

In [9]:
tf = termcounts/doctotals

In [10]:
doccounts = (termcounts>0).sum(axis=1)

In [11]:
idf = np.log(0.01 + numdocs/doccounts)

In [12]:
tfidf = tf.mul(idf, axis=0).fillna(0)

In [13]:
#tfidf.to_csv("data/tfidf.csv", sep="\t", index=None)

In [14]:
doccounts = doccounts.reset_index()
doccounts.columns = ['token','df']
doccounts.to_csv("data/df.csv", sep="\t", index=None)

In [15]:
yearname = dframe[['year','name']].drop_duplicates()

In [16]:
yearname.to_csv("data/yearname.csv", sep="\t", index=None)

In [17]:
seq = dframe[['token','year','seq']]

In [18]:
seq.to_csv('data/seq.csv', sep="\t", index=None)

In [19]:
termcounts.to_csv("data/termcounts.csv", sep="\t")

In [20]:
#tf.to_csv("data/tf.csv", sep="\t")

In [21]:
#idfframe = pd.DataFrame(idf).reset_index()
#idfframe.columns = ['lem','idf']
#idfframe.to_csv("data/idf_.csv", sep="\t", index=None)